In [66]:
import pandas as pd

# Read the Excel file (no need to specify custom na_values)
df = pd.read_csv('../../data_set/monthlyInvestigation/monthly_investigations.csv')

df.drop(columns=['HCO3 - pre HD (mmol/L)', 'HCO3 -post HD (mmol/L)', 'HbA1C (%)', 'PTH', 'Serum ferritin' , 'Serum iron', 'TSAT', 'Vit D' ,'blood  picture', 'Serum Na Post-HD (mmol/L)', 'S Phosphate (mmol/L)'], inplace=True)

# Columns to exclude
excluded_columns = ['Subject_ID', 'Month']

# Columns to clean
columns_to_clean = [col for col in df.columns if col not in excluded_columns]

# Convert non-numeric values to NaN in the target columns
df[columns_to_clean] = df[columns_to_clean].apply(pd.to_numeric, errors='coerce')

# Drop rows where all target columns are zero (ignoring NaN)
df = df[~(df[columns_to_clean].fillna(0) == 0).all(axis=1)]


In [67]:
# Interpolation logic per patient
def interpolate_or_fill(group):
    for col in columns_to_clean:
        series = group[col]
        non_null_count = series.notna().sum()
        
        if non_null_count == 0:
            # All values missing → leave as is (all NaN)
            continue
        elif non_null_count == 1:
            # Only one value → fill entire column with that single value
            group[col] = series.fillna(series.dropna().iloc[0])
        else:
            # Normal interpolation
            group[col] = series.interpolate(method='linear', limit_direction='both')
    
    return group

# Apply per patient
df[columns_to_clean] = (
    df.groupby('Subject_ID', group_keys=False)[columns_to_clean]
      .apply(interpolate_or_fill)
      .round(2)
)

df.dropna(inplace=True)
df

,Subject_ID,Month,Albumin (g/L),BU - post HD,BU - pre HD,Hb (g/dL),S Ca (mmol/L),SCR- post HD (µmol/L),SCR- pre HD (µmol/L),Serum K Post-HD (mmol/L),Serum K Pre-HD (mmol/L),Serum Na Pre-HD (mmol/L),UA (mg/dL)
0,RHD_THP_001,1/1/2024,28.6,0.26,13.50,11.0,2.25,96.91,538.00,2.40,4.30,136.0,392.0
1,RHD_THP_001,2/1/2024,37.7,0.38,14.14,13.1,2.23,97.37,513.60,2.90,4.32,136.0,408.0
2,RHD_THP_001,3/1/2024,40.6,5.85,12.30,12.8,2.24,276.00,560.00,3.90,4.75,141.4,404.0
3,RHD_THP_001,4/1/2024,35.8,6.30,18.90,12.2,1.97,325.40,689.00,4.40,4.20,138.4,487.0
4,RHD_THP_001,5/1/2024,36.9,6.75,14.79,12.9,2.53,374.80,601.93,4.90,3.80,138.2,336.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,RHD_THP_045,8/1/2024,31.4,3.70,8.29,9.6,2.30,162.00,402.50,3.13,3.78,134.6,365.0
536,RHD_THP_045,9/1/2024,32.6,0.27,11.37,10.5,2.54,63.80,466.00,2.36,4.20,134.0,345.0
537,RHD_THP_045,10/1/2024,32.6,0.27,8.50,11.3,2.54,63.80,397.00,2.36,4.20,134.0,345.0
538,RHD_THP_045,11/1/2024,32.1,0.27,15.60,11.0,2.49,128.90,440.00,2.36,4.05,134.0,352.0


In [68]:
df['Month'] = pd.to_datetime(df['Month'])
df.sort_values(['Subject_ID', 'Month'], inplace=True)
df

,Subject_ID,Month,Albumin (g/L),BU - post HD,BU - pre HD,Hb (g/dL),S Ca (mmol/L),SCR- post HD (µmol/L),SCR- pre HD (µmol/L),Serum K Post-HD (mmol/L),Serum K Pre-HD (mmol/L),Serum Na Pre-HD (mmol/L),UA (mg/dL)
0,RHD_THP_001,2024-01-01,28.6,0.26,13.50,11.0,2.25,96.91,538.00,2.40,4.30,136.0,392.0
1,RHD_THP_001,2024-02-01,37.7,0.38,14.14,13.1,2.23,97.37,513.60,2.90,4.32,136.0,408.0
2,RHD_THP_001,2024-03-01,40.6,5.85,12.30,12.8,2.24,276.00,560.00,3.90,4.75,141.4,404.0
3,RHD_THP_001,2024-04-01,35.8,6.30,18.90,12.2,1.97,325.40,689.00,4.40,4.20,138.4,487.0
4,RHD_THP_001,2024-05-01,36.9,6.75,14.79,12.9,2.53,374.80,601.93,4.90,3.80,138.2,336.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,RHD_THP_045,2024-08-01,31.4,3.70,8.29,9.6,2.30,162.00,402.50,3.13,3.78,134.6,365.0
536,RHD_THP_045,2024-09-01,32.6,0.27,11.37,10.5,2.54,63.80,466.00,2.36,4.20,134.0,345.0
537,RHD_THP_045,2024-10-01,32.6,0.27,8.50,11.3,2.54,63.80,397.00,2.36,4.20,134.0,345.0
538,RHD_THP_045,2024-11-01,32.1,0.27,15.60,11.0,2.49,128.90,440.00,2.36,4.05,134.0,352.0


In [69]:
df.to_excel('cleaned_monthly_investigations.xlsx', index=False)